# **02 - Data Exploration and Cleaning**

## Objectives

* Clean the dataset to ensure it is ready for analysis and modeling.
* Handle missing values, outliers, and inconsistent data to improve data quality.
* Optimize data types for better performance and memory efficiency.
* Ensure data integrity by addressing issues with file formats and missing values.
* Save the cleaned dataset for further analysis and modeling.

## Inputs

* The raw dataset containing house price records: `inputs/datasets/raw/house_prices_records.csv`.

## Outputs

* An updated cleaned dataset saved as: `outputs/datasets/cleaned/house_prices_cleaned.parquet`.
  - The `.parquet` format was chosen to preserve data types and handle missing values more reliably than `.csv`.

## Additional Comments

Due to issues with the `GarageFinish` column, where no missing values were present before saving but 235 missing values appeared when the saved file was loaded as a `.csv`, we have decided to use the `.parquet` format instead. 

The `.parquet` format preserves data types and handles missing values more reliably than `.csv`, ensuring the integrity of the cleaned dataset. This is crucial for maintaining data quality and consistency during analysis and modeling.

---

## Change Working Directory

Ensure the working directory is set to the project root for consistent file paths. This ensures that all file paths work correctly, regardless of where the notebook is executed.

In [14]:
import os
current_dir = os.getcwd()
current_dir

'd:\\Projects'

In [15]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [16]:
current_dir = os.getcwd()
current_dir

'd:\\'

---

## 1. Load Data

In [18]:
# Import Pandas for data manipulation
import pandas as pd

# Create DataFrame of raw dataset containing house prices
df_prices = pd.read_csv("inputs/datasets/raw/house_prices_records.csv")

# Display the amount of rows/columns and first few rows of the dataset
print("Shape of the dataset:", df_prices.shape)
df_prices.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'inputs/datasets/raw/house_prices_records.csv'

---

## 2. Data Exploration

### 2.1 Identify Missing Values
In this step, we aim to understand the scope of missing data in the dataset. This is important because:
- Missing values can introduce bias or errors in the analysis and modeling process.
- Identifying the most affected columns helps prioritize cleaning efforts.

#### Steps:
1. Identify columns with missing values.
2. Sort the columns in descending order to highlight the most affected variables.
3. Create a summary table showing:
   - The number of missing values.
   - The percentage of missing values relative to the dataset size.
   - The data type of each column.

In [ ]:
# Calculate the number of missing values for each column
missing_values = df_prices.isnull().sum()

# Filter columns with missing values and sort them in descending order
# This helps prioritize variables with the most missing data
missing_data = missing_values[missing_values > 0].sort_values(ascending=False)

# Create a DataFrame to summarize missing data
missing_data_df = pd.DataFrame({
    "Column": missing_data.index,
    "Missing Values": missing_data.values,
    "Percentage": (missing_data.values / len(df_prices) * 100).round(2),
    "Datatype": [df_prices[col].dtype for col in missing_data.index]
})

# Display the missing data summary
missing_data_df

### 2.2 Generate Profiling Report
Generate a detailed report for columns with missing values.

In [17]:
# Import YData Profiling library for exploratory data analysis
from ydata_profiling import ProfileReport

# Create a list of variables with missing data
vars_with_missing_data = missing_data.index.tolist()

# Generate profile report and display in notebook
if vars_with_missing_data:
    profile = ProfileReport(df=df_prices[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    # Display a message indicating no missing data
    print("There are no variables with missing data")

NameError: name 'missing_data' is not defined

---

## 3. Data Cleaning

### 3.1 Handling Missing Values
In this step, we address missing values to ensure the dataset is complete and ready for analysis. The following methods are used:
- **Numeric columns**: Missing values are imputed with the median to avoid skewing the data.
- **Categorical columns**: Logical values such as `'None'` or `'No'` are used to fill missing values, ensuring consistency.

In [ ]:
# Impute missing values in numeric columns with the median
df_prices['LotFrontage'].fillna(df_prices['LotFrontage'].median(), inplace=True)
df_prices['BedroomAbvGr'].fillna(df_prices['BedroomAbvGr'].median(), inplace=True)
df_prices['MasVnrArea'].fillna(df_prices['MasVnrArea'].median(), inplace=True)

# Fill missing values for categorical variables
df_prices['BsmtExposure'].fillna('No', inplace=True)
df_prices['BsmtFinType1'].fillna('Unf', inplace=True)
df_prices['GarageFinish'].fillna('None', inplace=True)

# Fill missing values for numeric variables
df_prices['2ndFlrSF'].fillna(0, inplace=True)
df_prices['GarageYrBlt'].fillna(0, inplace=True)

# Display the count of missing values after imputation
print(df_prices.isnull().sum())

### 3.2 Dropping columns
Dropping two columns since missing value is above 80%.

In [ ]:
df_prices.drop(columns=['EnclosedPorch', 'WoodDeckSF'], inplace=True)

---

## 4. Split Dataset

### 4.1 Create New Directory and Save Cleaned Dataset
Switch file format from `.csv` to `parquet` due to issues with missing values in `GarageFinish`.

In [ ]:
# Create output directory for cleaned data
try:
    os.makedirs(name='outputs/datasets/cleaned', exist_ok=True)
except Exception as e:
    print(e)

# Save cleaned DataFrame in Parquet format
df_prices.to_parquet("outputs/datasets/cleaned/house_prices_cleaned.parquet", index=False)

### 4.2 Split Data into Train and Test Set
To prepare the dataset for modeling, we split it into training and testing sets:
- **Training set**: Used to train the machine learning model, ensuring the model learns patterns in the data.
- **Testing set**: Used to evaluate the model's performance on unseen data, providing an unbiased estimate of its accuracy.

In [ ]:
# import library
from sklearn.model_selection import train_test_split

# Load cleaned dataset
df_cleaned = pd.read_parquet("outputs/datasets/cleaned/house_prices_cleaned.parquet")

# Split data into training and testing sets
TrainSet, TestSet = train_test_split(
    df_cleaned,
    test_size=0.2,
    random_state=42)

# Display row and columns of training and test set
print(f"Train set shape: {TrainSet.shape}")
print(f"Test set shape: {TestSet.shape}")

### 4.3 Check Missing Values and Display Results

In [ ]:
# Train set
print(f"Missing values in TrainSet:\n{TrainSet.isnull().sum()}")

# Test set
print(f"Missing values in TestSet:\n{TestSet.isnull().sum()}")

---

## 5. Visualize Data Cleaning Effects

The `DataCleaningEffect` function is used to:
1. **Verify the cleaning process**: Ensure that missing values have been handled correctly and that the data cleaning hasn't introduced any unexpected changes.
2. **Compare original vs cleaned data**: See how the distributions of variables have changed after cleaning.
3. **Document the process**: Provide clear visual evidence of the cleaning steps for transparency and reproducibility.

### Results
The visualizations below show the impact of data cleaning on selected variables. Key observations include:
- **`LotFrontage`**: Missing values were imputed with the median, resulting in a smoother distribution.
- **`GarageFinish`**: Missing values were filled with `'None'`, ensuring consistency in the categorical data.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Theme
sns.set_theme(style="darkgrid")

# Function to visualize the effect of data cleaning
def DataCleaningEffect(df_original, df_cleaned, variables_applied_with_method):
    """
    Visualize the impact of data cleaning on selected variables.

    Parameters:
    - df_original (DataFrame): The original dataset before cleaning.
    - df_cleaned (DataFrame): The cleaned dataset after processing.
    - variables_applied_with_method (list): List of variables to analyze.

    Returns:
    - None: Displays bar charts for categorical variables and histograms for numeric variables.
    """
    flag_count = 1

    # Identify categorical variables
    categorical_variables = df_original.select_dtypes(exclude=['number']).columns

    # Loop through every variable in the list
    for var in variables_applied_with_method:
        print(f"*** Distribution Effect Analysis After Data Cleaning Method on variable: {var}")

        if var in categorical_variables:
            # For categorical variables, create a bar chart
            df1 = pd.DataFrame({"Type": "Original", "Value": df_original[var]})
            df2 = pd.DataFrame({"Type": "Cleaned", "Value": df_cleaned[var]})
            dfAux = pd.concat([df1, df2], axis=0)
            dfAux.reset_index(drop=True, inplace=True) 

            fig, axes = plt.subplots(figsize=(8, 4))
            sns.countplot(
                data=dfAux, 
                x="Value", 
                hue="Type", 
                palette=sns.color_palette("Spectral", n_colors=2) 
                )
            axes.set_title(f"Distribution Plot {flag_count}: {var}")
            plt.xticks(rotation=90)
            plt.legend()
            plt.show()
            print(f"*** Bar plot for categorical variable: {var}")

        else:
            # For numeric variables, create histograms
            fig, axes = plt.subplots(figsize=(8, 4))
            sns.histplot(
                data=df_original, 
                x=var, 
                color=sns.color_palette("Spectral")[4],  
                label='Original', 
                kde=True, 
                element="step", 
                ax=axes
                )
            sns.histplot(
                data=df_cleaned, 
                x=var, 
                color=sns.color_palette("Spectral")[5],  
                label='Cleaned', 
                kde=True, 
                element="step", 
                ax=axes
                )
            axes.set_title(f"Distribution Plot {flag_count}: {var}")
            plt.legend()
            plt.show()
            print(f"*** Histogram for numerical variable: {var}")

        plt.close(fig)
        flag_count += 1


# Variables to verify
variables_to_verify = ['LotFrontage', 'BedroomAbvGr', 'GarageFinish', 'MasVnrArea']

# Call function to verify cleaning process
DataCleaningEffect(df_original=df_prices, 
                   df_cleaned=TrainSet, 
                   variables_applied_with_method=variables_to_verify)

---

## Conclusion and Next Steps

### Conclusions
The data cleaning process was successfully completed, ensuring the dataset is ready for further analysis and modeling. Key steps and outcomes include:

1. **Handling Missing Values**:
   - Missing values in numeric columns (e.g., `LotFrontage`, `BedroomAbvGr`, `MasVnrArea`) were imputed using the median to preserve data integrity.
   - Missing values in categorical columns (e.g., `GarageFinish`, `BsmtExposure`) were filled with logical values such as `'None'` or `'No'`.

2. **Dropping Irrelevant Columns**:
   - Columns with a high percentage of missing values (`EnclosedPorch`, `WoodDeckSF`) were removed to streamline the dataset and improve model performance.

3. **Optimizing Data Types**:
   - Categorical columns were converted to the `category` data type, reducing memory usage and improving computational efficiency.

4. **Splitting Data**:
   - The cleaned dataset was split into training and testing sets (80/20 split) to prepare for modeling.
   - Both sets were verified to ensure no missing values remain.

5. **Verification**:
   - Visualizations were used to compare the original and cleaned datasets, confirming that the cleaning process was effective and introduced no unexpected changes.

### Next Steps: Correlation Study
1. **Formulate Hypotheses**:
   - Identify key attributes likely to influence the target variable (`SalePrice`), such as `OverallQual`, `GrLivArea`, and `GarageFinish`.
   - Develop hypotheses to test the relationships between these attributes and `SalePrice`.

2. **Analyze Relationships**:
   - Perform a correlation study to quantify the strength of relationships between selected attributes and `SalePrice`.
   - Use statistical methods to validate the significance of these relationships.

3. **Visualize Insights**:
   - Create visualizations (e.g., heatmaps, scatterplots, boxplots) to explore and present the relationships between attributes and `SalePrice`.
   - Highlight key predictors that will be used in the predictive model.

4. **Prepare for Feature Engineering**:
   - Based on the insights from the correlation study, identify attributes that may require transformation or scaling for modeling.
   - Plan feature engineering steps to enhance model performance.

### Save Files
The cleaned and split datasets were saved in the `outputs/datasets/cleaned/` directory for future use:
- **Train Set**: `train_set.parquet`
- **Test Set**: `test_set.parquet`

In [ ]:
# Save Train Set
TrainSet.to_parquet("outputs/datasets/cleaned/train_set.parquet", index=False)

# Save Test Set
TestSet.to_parquet("outputs/datasets/cleaned/test_set.parquet", index=False)